# ML model

In [1]:
#pip install pymorphy2

In [2]:
import os
import string
import annoy
import json
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer


import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [3]:
products = pd.read_csv('../input/products/ProductsDataset.csv')
products

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...


In [4]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    #spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
products['title'] = products['title'].apply(preprocess_txt)

In [6]:
#Add possible requests that do not contain products enquiries
sentences = []
c = 0
with open("../input/otvety/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 125000:
            break

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


0it [00:00, ?it/s]

In [7]:
sentences[:5]

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка']]

In [8]:
# Some results are just emplty lists
i = 0
for item in sentences:
    if item == []:
        sentences.pop(i)
    i += 1

In [9]:
len(sentences)

104574

In [10]:
# Combine products enquiries with label = 1 and other enquiries with label = 0
products['label'] = 1
products.sample(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,label
7546,[брюки],Брюки для девочек,5bdbdb32bd36c06af8597d08,22.0,2216,"{'sostojanie_garderob': 'Б/у', 'detskie_pol': ...",http://cache3.youla.io/files/images/360_360/5b...,1
17166,[свитер],NaN,5b6179ba80e08e0b085562ce,9.0,905,{'zhenskaya_odezhda_razmer': '40-42 (XS)'},http://cache3.youla.io/files/images/360_360/5b...,1
8418,"[комплект, кроватка, новый]",Продаю новый комплект постельного белья в детс...,58a4a93bd53f3d804bfb866d,3.0,317,{'detskie_mebel_tip': 'Кроватки'},http://cache3.youla.io/files/images/360_360/58...,1
34935,"[бутылочка, больший, авента]","Avent natural, 3 большие бутылочки с переменны...",5beab21622a44931077348db,3.0,311,"{'kormlenie_pitanie_tip': 'Бутылочки, ниблеры'}",http://cache3.youla.io/files/images/360_360/5b...,1
13006,"[шезлонг, баунсера, люлька, tiny, love]",Продаю музыка вибрация,5a083988c15ae35c0836e59b,3.0,317,"{'detskie_mebel_tip': 'Качели, шезлонги'}",http://cache3.youla.io/files/images/360_360/5b...,1


In [11]:
other_enquiries = pd.DataFrame(np.array(sentences), columns=['text'])
other_enquiries['label'] = 0
other_enquiries.sample()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


,text,label
30508,"[послезавтра, выспаться, наверное]",0


In [12]:
prod_data = products[['title', 'label']]
prod_data = prod_data.rename(columns={'title': 'text'})

In [13]:
data = pd.concat([prod_data, other_enquiries])

In [14]:
data.sample(10)

,text,label
16113,"[консилиум, 5, мужик, единогласно]",0
27295,"[кроссовок, осень]",1
84161,"[поверье, умирающий, ведьма, умереть, передать...",0
71202,"[хороший, выбрать, соперник, игра, место, случ...",0
1774,"[аэс, ружьё, висеть, акт, спектакль, выстрелит...",0
72923,[удоюный],0
24906,"[сандалия, орлёнок, кожа, мд, р, 26]",1
100179,"[мороз, синий, нос, сильно, пить]",0
59114,[httpwwwskypecomintlrudownloadskypewindowscmmm...,0
26493,[ветровка],1


In [15]:
data.to_csv('all_enquiries.csv', index=False)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140122 entries, 0 to 104573
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    140122 non-null  object
 1   label   140122 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ MB


In [17]:
data['text'] = data['text'].apply(lambda x: ' '.join(x))

In [18]:
vectorizer = TfidfVectorizer()
vectorizer.fit(data['text'])

TfidfVectorizer()

In [19]:
with open('vectorizer.pk', 'wb') as fin:
    pickle.dump(vectorizer, fin)

In [20]:
X = data['text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=None)

In [21]:
y_test.value_counts()

0    20860
1     7165
Name: label, dtype: int64

In [22]:
X_train_vec = vectorizer.transform(X_train)

In [39]:
lg_model = LogisticRegression(random_state=42, solver='liblinear').fit(X_train_vec, y_train)

In [40]:
X_test_vec = vectorizer.transform(X_test)
prediction = lg_model.predict(X_test_vec)

In [41]:
accuracy = (prediction == y_test).mean()
accuracy

0.9696699375557538

In [42]:
svm_model = LinearSVC(random_state=42).fit(X_train_vec, y_train)

In [43]:
X_test_vec = vectorizer.transform(X_test)
prediction = svm_model.predict(X_test_vec)

In [44]:
accuracy = (prediction == y_test).mean()
accuracy

0.9793398751115076

In [45]:
# Save the best model
import pickle
with open('svm_model_pkl', 'wb') as files:
    pickle.dump(svm_model, files)